# Worksheet 07

Name:  Sangjoon Lee
UID: U79516048

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.494396875155613, 4.833571493442839, 4.613500433607381, 4.770597222100016, 5.439457431491844, 4.0711884686318305, 4.9994004390514135, 4.398022247992161, 4.286591671862009, 3.905665712836324]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[8.467121871289015, 7.438763083177053, 8.673007558285963, 8.021705518474782, 7.96329515791688, 6.987351659796287, 8.007747507134848, 6.711000888164377, 7.925591302991277, 6.528784211232736]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[6.528784211232736, 3.905665712836324, 4.286591671862009, 7.925591302991277, 4.398022247992161, 4.9994004390514135, 6.711000888164377, 8.007747507134848, 4.0711884686318305, 5.439457431491844]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

What are the parameters of a GMM?
P(c1) = P(c2) = 1/2
mu_1 = 5
mu_2 = 8
var_1 = var_2 = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.905665712836324, 4.286591671862009, 4.398022247992161, 4.9994004390514135, 4.0711884686318305, 5.439457431491844]
[6.528784211232736, 7.925591302991277, 6.711000888164377, 8.007747507134848]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 4.516720995310931,  mean_2 = 7.293280977380809
var_1 = 0.28722644412679377,  var_2 = 0.45844604444028814


Probability of cluters is spot-on.
Mean is very similar, and variance is quite different. especially for var_2.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  6.528784211232736
probability of observing that point if it came from cluster 0 =  3.0677538532990084e-11
probability of observing that point if it came from cluster 1 =  0.21665649353188005
point =  3.905665712836324
probability of observing that point if it came from cluster 0 =  0.14450492001768014
probability of observing that point if it came from cluster 1 =  1.2102139032726803e-12
point =  4.286591671862009
probability of observing that point if it came from cluster 0 =  1.0076041297518603
probability of observing that point if it came from cluster 1 =  3.9760144188593246e-10
point =  7.925591302991277
probability of observing that point if it came from cluster 0 =  3.6009896863569027e-31
probability of observing that point if it came from cluster 1 =  0.33615363164439915
point =  4.398022247992161
probability of observing that point if it came from cluster 0 =  1.2752657332599209
probability of observing that point if it came from cluster 1 =  1.9007693202871224e-09
po

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [ sum(map(lambda x: x[0] * (x[1] - mean[0])**2, zip(prob_c0_x, data))) , sum(map(lambda x: x[0] * (x[1] - mean[1])**2, zip(prob_c1_x, data))) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.5979928931522054,  P(C_2) = 0.4020071068477947
mean_1 = 4.513624379329665,  mean_2 = 7.284024677827092
var_1 = 1.7062158439216493,  var_2 = 1.902428674262626


The probability and mean of each clusters were relatively the same.
There is some difference in the variance, however.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  6.528784211232736
probability of observing that point if it came from cluster 0 =  0.11640505569937264
probability of observing that point if it came from cluster 1 =  0.19381146865219503
point =  3.905665712836324
probability of observing that point if it came from cluster 0 =  0.21943517247289832
probability of observing that point if it came from cluster 1 =  0.04333359585087935
point =  4.286591671862009
probability of observing that point if it came from cluster 0 =  0.2317562144869652
probability of observing that point if it came from cluster 1 =  0.06060973869543799
point =  7.925591302991277
probability of observing that point if it came from cluster 0 =  0.03166093341775802
probability of observing that point if it came from cluster 1 =  0.19810981738779018
point =  4.398022247992161
probability of observing that point if it came from cluster 0 =  0.2332809484260142
probability of observing that point if it came from cluster 1 =  0.06635548788685817
point =  4.999400

The previous soft assignments were much dominant in one cluster than another (either 99% or 0%).
The new assignments are much more realistic and shifts between 40% ~ 80% probabilyt of seeing a point from a speicific cluster.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [10]:
for p in probs:
    # if it has greater probability, assign it to that cluster
    if p[1] > p[2]:
        kmeans.labels_[data.index(p[0])] = 0
    else:
        kmeans.labels_[data.index(p[0])] = 1

kmeans.labels_

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0], dtype=int32)

### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      1       |
| A  C |      0       |
| A  D |      0       |
| A  E |      0       |
| B  C |      1       |
| B  D |      0       |
| B  E |      0       |
| C  D |      0       |
| C  E |      0       |
| D  E |      1       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

$\frac{(n)(n-1)}{2}$

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36 disagreements

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Cluster 0: 3 agreements
Cluster 1: 1 agreement
Cluster 2: 6 agreements

Total 10 agreemeents.
Total 26 disagreements.

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

For every unique pairs of points, check if the two points share a cluster in every clustering.
If they do, return 1. If they share no clusters in every clustring, also return 1.
Else, return 0.